In [10]:
from akashiwo_data_scrape import *
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
# speacies id
spicies_ids = [45, 3]
spicies_names = ["Diaton.csv", "shtonela.csv"]

for id, names in zip(spicies_ids, spicies_names):
    pull_data(id, "2011/5/01", "2023/10/11", names)

In [96]:
colum_names = ['lat', 'lng', 'speciesValueAM', 'speciesValuePM', 
               'saisuiValueAM', 'saisuiValuePM', 'speciesNameKana', 
               'maxvalue', 'speciesId', 'icon_size']

for name in spicies_names:
    remove_colum(colum_names, name)

In [15]:
data = pd.read_csv("shtonela.csv")
pointIds = data["pointId"]
gatherYMDs = data["gatherYMD"]

In [30]:
main_data = {}
coordinate_data = {}
headers_to_skip_main = ["確定値／速報値", "事業・調査名"]
i = 0
for pointId, gatherYMD in tqdm(zip(pointIds, gatherYMDs), desc="scraping", unit="requests"):
    html_data = requests.get(f"https://akashiwo.jp/private/akashiwoListInit.php?qpoint_id={str(pointId)}&qspecies_id=3&qgather_ymd_s=&qgather_ymd_e={str(gatherYMD)}")
    html_data.encoding = 'utf-8'
    soup = BeautifulSoup(html_data.text, 'html.parser')
    tables = soup.find_all('table')

    times_to_duplicate = parse_main_table(tables[1], headers_to_skip_main, main_data)
    parse_coordinate_table(tables[0], coordinate_data, times_to_duplicate)
    if(i == 20): break
    i += 1

scraping: 3requests [00:00, 10.81requests/s]

{'採取日': ['2014-06-26', '2014-06-26'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['23.2', '21.4'], '塩分': ['33.15', '33.84'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['103', '98.2'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.39', '0.36'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['0', '0'], '珪藻類合計': ['1840', '1416.7'], 'コクロディニウム\u3000ポリクリコイデス': ['0', '0'], 'ヘテロカプサ\u3000サーキュラリスカーマ': ['0', '0'], 'カレニア\u3000ミキモトイ': ['0', '0'], 'ヘテロシグマ\u3000アカシオ': ['0', '0'], '備考': ['', '']}
{'採取日': ['2014-07-07', '2014-07-07'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['21.21', '20.28'], '塩分': ['33.5', '33.95'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['95.9', '90.8'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.75', '2.12'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['

scraping: 5requests [00:00, 11.16requests/s]

{'採取日': ['2015-06-29', '2015-06-29'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['21.98', '21.88'], '塩分': ['33.73', '33.88'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['101.9', '104.56'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.23', '0.07'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['1', '0'], 'シャットネラ合計値（自動計算）': ['1', '0'], '珪藻類合計': ['2930', '940'], 'コクロディニウム\u3000ポリクリコイデス': ['0', '0'], 'ヘテロカプサ\u3000サーキュラリスカーマ': ['0', '0'], 'カレニア\u3000ミキモトイ': ['2', '0'], 'ヘテロシグマ\u3000アカシオ': ['0', '0'], '備考': ['', '']}
{'採取日': ['2015-07-06', '2015-07-06'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['20.52', '19.8'], '塩分': ['33.74', '34'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['94.8', '87.79'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['1.15', '2.43'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': [

scraping: 9requests [00:00, 11.55requests/s]

...

{'採取日': ['2015-07-06', '2015-07-06'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['19.94', '19.85'], '塩分': ['33.87', '33.97'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['93.97', '90.39'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['1.56', '2.15'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['0', '0'], '珪藻類合計': ['810', '130'], 'コクロディニウム\u3000ポリクリコイデス': ['0', '0'], 'ヘテロカプサ\u3000サーキュラリスカーマ': ['0', '0'], 'カレニア\u3000ミキモトイ': ['0', '1'], 'ヘテロシグマ\u3000アカシオ': ['0', '0'], '備考': ['', '']}
{'採取日': ['2015-07-29', '2015-07-29'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['26.14', '24.08'], '塩分': ['33.62', '33.94'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['105.52', '100.34'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.02', '0.13'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算

scraping: 17requests [00:01, 10.88requests/s]

{'採取日': ['2015-08-10', '2015-08-10'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['26.89', '25.07'], '塩分': ['33.95', '33.88'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['108.61', '109.29'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0', '0'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['0', '0'], '珪藻類合計': ['440', '300'], 'コクロディニウム\u3000ポリクリコイデス': ['0', '0'], 'ヘテロカプサ\u3000サーキュラリスカーマ': ['0', '0'], 'カレニア\u3000ミキモトイ': ['2', '1'], 'ヘテロシグマ\u3000アカシオ': ['0', '0'], '備考': ['', '']}
{'採取日': ['2014-06-26', '2014-06-26'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['23.16', '22.78'], '塩分': ['32.93', '33.71'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['104.3', '100.6'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.2', '0.16'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['0

scraping: 20requests [00:01, 10.47requests/s]

{'採取日': ['2015-06-29', '2015-06-29'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['21.99', '21.82'], '塩分': ['33.64', '33.88'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['103.16', '101.58'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['0.01', '0.03'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）': ['0', '0'], '珪藻類合計': ['560', '270'], 'コクロディニウム\u3000ポリクリコイデス': ['0', '0'], 'ヘテロカプサ\u3000サーキュラリスカーマ': ['0', '0'], 'カレニア\u3000ミキモトイ': ['0', '0'], 'ヘテロシグマ\u3000アカシオ': ['0', '0'], '備考': ['', '']}
{'採取日': ['2015-07-06', '2015-07-06'], '採取時刻': ['', ''], '採水層': ['表層', ''], '採水層（ｍ）': ['0.5', '10'], '水温（℃）': ['19.95', '19.61'], '塩分': ['33.81', '34.04'], '比重(σ15)': ['', ''], '溶存酸素(mg/L)': ['', ''], '溶存酸素(％)': ['93.43', '89.94'], 'プランクトン沈殿量(ml/m³)': ['', ''], '溶存態無機窒素(µmol/L)': ['1.84', '2.9'], 'シャットネラ\u3000アンティカ': ['0', '0'], 'シャットネラ\u3000オバータ': ['0', '0'], 'シャットネラ属': ['0', '0'], 'シャットネラ合計値（自動計算）

In [29]:
merged_data = {**coordinate_data, **main_data}
df = pd.DataFrame(merged_data)
df.to_csv("hehe.csv", index=False)